In [1]:
import pandas as pd
import numpy as np


In [2]:
import os


In [3]:
os.getcwd()

'/home/ilya/1_Study/Diploma/Input-Output_tables'

In [4]:

# Изменяем текущую директорию на 'data'
os.chdir("data")

# Проверяем, что директория изменилась
print("Текущая директория изменена на:", os.getcwd())

Текущая директория изменена на: /home/ilya/1_Study/Diploma/Input-Output_tables/data


## Data preparation

In [7]:
file_name = '2quadrant_aggr_normalized_by2q.xlsx'
# Читаем конкретный лист по имени
start_year = 2017
end_year = 2023

dict_2q = dict()
for year in range(start_year, end_year+1):
    df_year =  pd.read_excel(file_name, sheet_name=str(year), index_col=0)
    dict_2q[year] = df_year

In [18]:
tmp = dict_2q[2020]
tmp.index

Index(['Промежуточное потребление', 'домашние хозяйства', 'Гос управление',
       'Валовое накопление', 'Экспорт', 'ДХ и промежуточное потребление'],
      dtype='object')

In [9]:
dict_2q[2020]

,Использование продукции в отраслях экономики,Расходы на конечное потребление домашних хозяйств,"Расходы на конечное потребление органов государственного управления, всего",Расходы на конечное потребление НКООДХ,"Валовое накопление, всего",Экспорт товаров и услуг
Промежуточное потребление,0.618828,0.113365,0.019780,0.000000,1.059491e-01,0.142077
домашние хозяйства,0.170565,0.694172,0.008727,0.024414,4.678773e-02,0.055334
Гос управление,0.003584,0.286807,0.707999,0.000930,2.392609e-05,0.000656
Валовое накопление,0.178214,0.115899,0.002384,0.000000,6.696174e-01,0.033886
Экспорт,0.129835,0.000000,0.000000,0.000000,8.320979e-02,0.786955
ДХ и промежуточное потребление,0.503734,0.441757,0.001024,0.000000,6.763171e-07,0.053484


In [10]:

# Определяем категории для графиков
categories = {
    'household': 'Конечное потребление д/х, доли в комплексах',
    'government': 'Конечное потребление государства, доли в комплексах',
    'capital': 'Валовое накопление основного капитала, доли в комплексах',
    'export': 'Экспорт, доли в комплексах'
}

# Определяем цвета для секторов
sector_colors = {
    'Обрабатывающий': '#1F77B4',
    'Экспортирующий': '#FF7F0E',
    'Инфраструктурный': '#2CA02C',
    'Услуги': '#D62728',
    'Финансы': '#9467BD',
    'Импорт': '#8C564B'
}

In [11]:

import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

In [20]:
# Предположим структуру данных, где строки - это сектора экономики, а столбцы - это категории расходов
# Функция для создания графика
def create_stacked_bar_chart(data_frames, category_num, title):
    """
    Создает график с накопленными столбцами для определенной категории расходов
    
    Parameters:
    data_frames (dict): Словарь с данными фреймов по годам
    category_name (str): Название категории расходов (столбец в данных)
    title (str): Заголовок для графика
    """
    plt.figure(figsize=(12, 6))
    
    # Предполагаем, что у нас есть несколько листов/лет данных
    # Получаем годы из ключей словаря
    years = list(data_frames.keys())
    years.sort()  # Сортируем годы
    
    sectors = data_frames[years[0]].index
    # Создаем массив данных для каждого года и сектора
    all_data = []
    
    for year in years:
        tmp = data_frames[year]
        all_data += tmp.iloc[:, category_num]

    # Транспонируем данные для корректного построения
    all_data = np.array(all_data).T

    # Рисуем накопленную столбчатую диаграмму
    bottom = np.zeros(len(years))
    
    for i, sector in enumerate(sectors):
        color = sector_colors.get(sector, f'C{i}')  # Используем предопределенный цвет или цвет из палитры
        plt.bar(years, all_data[i], bottom=bottom, label=sector, color=color)
        bottom += all_data[i]
    
    # Настраиваем график
    plt.title(title)
    plt.xlabel('Год')
    plt.ylabel('Доля в комплексах')
    plt.ylim(0, 1.0)
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1.0))
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    
    # Возвращаем фигуру для дальнейшего использования
    return plt.gcf()

# Создаем и сохраняем графики для каждой категории расходов
for category_key, category_title in categories.items():
    try:
        fig = create_stacked_bar_chart(dict_2q, category_key, category_title)
        # Сохраняем график
        output_file = f"{category_key}_chart.png"
        fig.savefig(output_file, dpi=300, bbox_inches='tight')
        print(f"График сохранен в файл: {output_file}")
        plt.close(fig)  # Закрываем фигуру, чтобы освободить память
    except Exception as e:
        print(f"Ошибка при создании графика для {category_title}: {e}")
# Если стандартный код не создал графики, создаем демонстрационные
print("\nСоздание демонстрационных графиков...")

print("\nВсе графики созданы успешно!")

Ошибка при создании графика для Конечное потребление д/х, доли в комплексах: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types
Ошибка при создании графика для Конечное потребление государства, доли в комплексах: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types
Ошибка при создании графика для Валовое накопление основного капитала, доли в комплексах: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types
Ошибка при создании графика для Экспорт, доли в комплексах: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

Создание демонстрационных графиков...

Все графики созданы успешно!

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>